In [ ]:
import pandas as pd
import tqdm
import numpy as np
import json
import sklearn
import random
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
import pickle as pickle

In [ ]:
def set_seed(seed:int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_seed(42)

In [ ]:
def label_2_num(label):
  for idx, value in enumerate(label.unique()):
    label_2_num[value] = idx
    
def num_2_label(label):
  for idx, value in enumerate(label.unique()):
    num_2_label[idx] = value

In [ ]:
class RE_Dataset(torch.utils.data.Dataset):
  """ Dataset 구성을 위한 class."""
  def __init__(self, pair_dataset, labels):
    self.pair_dataset = pair_dataset
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)
  
def preprocessing_dataset(dataset):
  label_list = []
  for main, detail in zip(dataset['main'], dataset['detail']):
    label = f'{main}_{detail}'

    label_list.append(label)
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  out_dataset = pd.DataFrame({'url':dataset['url'],'context':dataset['context'], 'label':label_list})
  return out_dataset

def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(dataset)
  
  return dataset

def tokenized_dataset(context, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  tokenized_context = tokenizer(context,return_tensors="pt", padding=True, truncation=True, max_length=256, add_special_tokens=True,)
  return tokenized_context

In [ ]:
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = list(df['label'].unique())
    label_indices = list(range(len(label_list)))
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0

def klue_re_auprc(probs, labels):
    """KLUE-RE AUPRC (with no_relation)"""
    labels = np.eye(len(df['label'].unique()))[labels]

    score = np.zeros((len(df['label'].unique()),))
    for c in range(len(df['label'].unique())):
        targets_c = labels.take([c], axis=1).ravel()
        preds_c = probs.take([c], axis=1).ravel()
        precision, recall, _ = sklearn.metrics.precision_recall_curve(targets_c, preds_c)
        score[c] = sklearn.metrics.auc(recall, precision)
    return np.average(score) * 100.0

def compute_metrics(pred):
  """ validation을 위한 metrics function """
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  probs = pred.predictions

  # calculate accuracy using sklearn's function
  f1 = klue_re_micro_f1(preds, labels)
  auprc = klue_re_auprc(probs, labels)
  acc = accuracy_score(labels, preds) # 리더보드 평가에는 포함되지 않습니다.

  return {
      'micro f1 score': f1,
      'auprc' : auprc,
      'accuracy': acc,
  }

In [ ]:
MODEL_NAME = 'google-bert/bert-base-multilingual-uncased'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
df = load_data('data/reviewed_emotion_0308.csv')
df.head()

In [ ]:
label_2_num = {}
num_2_label = {}

for idx, value in enumerate(df['label'].unique()):
  label_2_num[value] = idx
  num_2_label[idx] = value

In [ ]:
label = [label_2_num[value] for value in list(df['label'])]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['context'], label, test_size=0.3)
# X_train, X_test, y_train, y_test = train_test_split(df['context'], label, test_size=0.3, stratift=label)

In [ ]:
train_context = tokenized_context = tokenized_dataset(X_train.to_list(), tokenizer)
test_context = tokenized_context = tokenized_dataset(X_test.to_list(), tokenizer)

In [ ]:
RE_train_dataset = RE_Dataset(train_context, y_train)
RE_eval_dataset = RE_Dataset(test_context, y_test)

In [ ]:
model_config =  AutoConfig.from_pretrained(MODEL_NAME)
model_config.num_labels = len(df['label'].unique())

model =  AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=model_config)
model.parameters
model.to(device)

In [ ]:
training_args = TrainingArguments(
  output_dir='results',          # output directory
  save_steps=500,                 # model saving step. # 나중에 100이나 500으로 수정
  num_train_epochs=20,              # total number of training epochs
  learning_rate=1e-5,               # learning_rate
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=16,   # batch size for evaluation
  warmup_steps=500,                # number of warmup steps for learning rate scheduler # 나중에 100이나 500으로 수정
  weight_decay=0.01,               # strength of weight decay
  evaluation_strategy='steps', # evaluation strategy to adopt during training
                              # `no`: No evaluation during training.
                              # `steps`: Evaluate every `eval_steps`.
                              # `epoch`: Evaluate every end of epoch.
  logging_steps=500,              # log saving step. # 나중에 100이나 500으로 수정
  eval_steps = 500,            # evaluation step. # 나중에 100이나 500으로 수정
  load_best_model_at_end = True
  )
trainer = Trainer(
  model=model,                         # the instantiated 🤗 Transformers model to be trained
  args=training_args,                  # training arguments, defined above
  train_dataset=RE_train_dataset,         # training dataset
  eval_dataset=RE_eval_dataset,             # evaluation dataset
  compute_metrics=compute_metrics         # define metrics function
)

In [ ]:
trainer.train()

# aihub

In [ ]:
def inference(model, tokenized_sent, device):
  """
    test dataset을 DataLoader로 만들어 준 후,
    batch_size로 나눠 model이 예측 합니다.
  """
  dataloader = DataLoader(tokenized_sent, batch_size=16, shuffle=False)
  model.eval()
  output_pred = []
  output_prob = []
  for i, data in enumerate(dataloader):
    with torch.no_grad():
      outputs = model(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
          token_type_ids=data['token_type_ids'].to(device)
          )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
  return np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

# active learning

In [ ]:
with open('data/aihub.json') as aihub:
  parsed_json = json.load(aihub)
  
aihub_context = [data['talk']['content']['HS01'] for data in parsed_json]
aihub_label = [0] * len(aihub_context)
aihub_df = pd.DataFrame({'context':aihub_context, 'label':aihub_label})

In [ ]:
training_args = TrainingArguments(
  output_dir='results',          # output directory
  save_steps=500,                 # model saving step. # 나중에 100이나 500으로 수정
  num_train_epochs=20,              # total number of training epochs
  learning_rate=1e-5,               # learning_rate
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=16,   # batch size for evaluation
  warmup_steps=500,                # number of warmup steps for learning rate scheduler # 나중에 100이나 500으로 수정
  weight_decay=0.01,               # strength of weight decay
  evaluation_strategy='steps', # evaluation strategy to adopt during training
                              # `no`: No evaluation during training.
                              # `steps`: Evaluate every `eval_steps`.
                              # `epoch`: Evaluate every end of epoch.
  logging_steps=500,              # log saving step. # 나중에 100이나 500으로 수정
  eval_steps = 500,            # evaluation step. # 나중에 100이나 500으로 수정
  load_best_model_at_end = True
  )


learning_count = 10
cut_count = len(aihub_label) // learning_count

for n in range(learning_count):
  print(f'{n + 1}번째 시작')
  aihub_tokenized_context = tokenized_dataset(aihub_df['context'].to_list(), tokenizer)
  aihub_label = aihub_df['label'].to_list()

  RE_test_dataset = RE_Dataset(aihub_tokenized_context, aihub_label)

  print('예측 시작')
  pred_answer, output_prob = inference(model, RE_test_dataset, device) # model에서 class 추론
  print('예측 종료')
  pred_answer_label = [num_2_label[value] for value in pred_answer]
  aihub_df['pred'] = pred_answer
  aihub_df['prob'] = output_prob
  aihub_df['prob'] = aihub_df['prob'].apply(lambda x: max(x))
  aihub_df = aihub_df.sort_values(by='prob', ascending=False)

  new_learning_data = aihub_df.iloc[:cut_count]
  aihub_df = aihub_df.iloc[cut_count:]

  tokenized_context = tokenized_dataset(new_learning_data['context'].to_list(), tokenizer)
  NEW_train_dataset = RE_Dataset(tokenized_context, new_learning_data['pred'].to_list())

  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=NEW_train_dataset,         # training dataset
    eval_dataset=RE_eval_dataset,             # evaluation dataset
    compute_metrics=compute_metrics         # define metrics function
  )
  
  print('학습 시작')
  trainer.train()
  print('학습 종료')
  print(f'{n + 1}번째 종료')